In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Import Dependancies

In [101]:
#!/usr/bin/env python3.11
!python --version

import mesa
print(f"mesa version: {mesa.__version__}")
import numpy as np
import pandas as pd
import seaborn as sns
import random

DEBUG = False

Python 3.12.6
mesa version: 3.1.5


# The Naming Game

text

In [3]:
#ideas for own unique approach:
# multiple objects and graph how time until vocab convergence changes?
# different lengths of words and graph difference there?
# limit amount of symbols for words?
# 

Create Random Word

In [4]:
from random import choice
from string import ascii_uppercase

def create_word(min_length=6, max_length=10):
    new_word = ''.join(choice(ascii_uppercase) for _ in range(np.random.randint(min_length, max_length)))
    return new_word

Create Agent

In [97]:
class NamingGameAgent(mesa.Agent):
    def __init__(self, model, m_objects=1):
        super().__init__(model)
        self._m_objects = m_objects
        self.vocabulary = dict()

    def choose_object_and_word(self):
        #chosen from range will always choose 0 index object value, so m_objects = 1 -> chosen object is 0
        chosen_object = np.random.randint(0, self._m_objects)
        
        if chosen_object in self.vocabulary.keys():
            chosen_word = random.choice(self.vocabulary[chosen_object])
        else:
            chosen_word = create_word()
            self.vocabulary[chosen_object] = [chosen_word]
        return chosen_object, chosen_word

    def recognize_word(self, chosen_object, chosen_word):
        if chosen_object not in self.vocabulary.keys():
            return False
        if chosen_word in self.vocabulary[chosen_object]:
            return True
        else:
            return False

    def agree_to_word(self, chosen_object, chosen_word):
        self.vocabulary[chosen_object] = [chosen_word]

    def add_word(self, chosen_object, chosen_word):
        if chosen_object not in self.vocabulary.keys():
            self.vocabulary[chosen_object] = [chosen_word]
        else:
            self.vocabulary[chosen_object].append(chosen_word)

    def retrieve_vocabulary(self):
        return [f"Object: {key} - words: {value}" for key, value in self.vocabulary.items()]
    
    def word_amount(self):
        return sum([len(word_list) for word_list in self.vocabulary.values()])

Create Model

In [98]:
from mesa.datacollection import DataCollector

class NamingGameModel(mesa.Model):
    def __init__(self, N=100, m_objects=1, seed=None):
        super().__init__(seed=seed)
        self.num_agents = N
        self.successful_interactions = 0
        self.datacollector = DataCollector(
            agent_reporters={"Vocabulary": NamingGameAgent.retrieve_vocabulary, 
                             "Word_amount": NamingGameAgent.word_amount},
            model_reporters={"Successful_interactions": "successful_interactions",
                             "Total_word_amount": "total_word_amount",
                             "Total_unique_word_amount": "total_unique_word_amount"}
        )
        #self.detailed_datacollector = DataCollector(
        #    model_reporters={"Successful_interactions": "successful_interactions",
        #                     "Total_word_amount": "total_word_amount",
        #                     "Total_unique_word_amount": "total_unique_word_amount"}
        #)
        self.m_objects = m_objects
        NamingGameAgent.create_agents(model=self, n=N, m_objects=m_objects)
    
    def step(self):
        if self.steps % 100 == 0:
            self.datacollector.collect(self)
        speaker, hearer = random.sample(self.agents, 2)
        if self.steps % 100 == 0 and DEBUG:
            if speaker.word_amount() != 0:
                print(speaker.word_amount())
                print(speaker.retrieve_vocabulary())
        chosen_object, chosen_word = speaker.choose_object_and_word()
        if hearer.recognize_word(chosen_object, chosen_word):
            self.successful_interactions += 1
            speaker.agree_to_word(chosen_object, chosen_word)
            hearer.agree_to_word(chosen_object, chosen_word)
        else:
            hearer.add_word(chosen_object, chosen_word)

    @property
    def total_word_amount(self):
        return sum([agent.word_amount() for agent in self.agents])

    @property
    def total_unique_word_amount(self):
        unique_words = set()
        for agent in self.agents:
            for word_list in agent.vocabulary.values():
                unique_words.update(word_list)
        return len(unique_words)


Run Model

In [99]:
def run_model(N, M, t, iteration):
    model = NamingGameModel(N, M)
    for i in range(t):
        #debug_vocab = model.agents.select(at_most=1).get("vocabulary")
        #if debug_vocab[0]:
        #    print(f"step {i}: {debug_vocab}")
        if i % int(t/10) == 0:
            print(f"iter: {iteration}, step: {i}")
            
        model.step()

    agent_data = model.datacollector.get_agent_vars_dataframe()
    model_data = model.datacollector.get_model_vars_dataframe()
    return agent_data, model_data

In [102]:
N = 1000
M = 1
t = 60000

RUNS = 1
print(int(RUNS/10))
single_run1 = np.random.randint(0, int(RUNS/10)+1)
single_run2 = int(RUNS - single_run1)

model_df = pd.DataFrame()
single_run1_agent_data, single_run1_model_data = pd.DataFrame(), pd.DataFrame()
single_run2_agent_data, single_run2_model_data = pd.DataFrame(), pd.DataFrame()
for i in range(RUNS):
    agent_data, model_data = run_model(N=N, M=M, t=t, iteration=i+1)
    if i == single_run1:
        single_run1_agent_data = agent_data
        single_run1_model_data = model_data
    if i == single_run2:
        single_run2_agent_data = agent_data
        single_run2_model_data = model_data
    
    if model_df.empty:
        model_df = model_data
    else:
        model_df = (model_df + model_data) / 2

model_df

0
iter: 1, step: 0
iter: 1, step: 6000
iter: 1, step: 12000
iter: 1, step: 18000
iter: 1, step: 24000
iter: 1, step: 30000
iter: 1, step: 36000
iter: 1, step: 42000
iter: 1, step: 48000
iter: 1, step: 54000


,Successful_interactions,Total_word_amount,Total_unique_word_amount
0,0,189,90
1,0,353,154
2,0,521,222
3,0,676,277
4,0,814,315
...,...,...,...
595,32267,1000,1
596,32367,1000,1
597,32467,1000,1
598,32567,1000,1


Datacollection Dataframes

In [103]:
single_run1_agent_data

Vocabulary  Word_amount
Step  AgentID                                                 
100   1                                        []            0
      2                                        []            0
      3                                        []            0
      4                                        []            0
      5        [Object: 0 - words: ['UIORIGLMN']]            1
...                                           ...          ...
60000 996        [Object: 0 - words: ['OFOLEXD']]            1
      997        [Object: 0 - words: ['OFOLEXD']]            1
      998        [Object: 0 - words: ['OFOLEXD']]            1
      999        [Object: 0 - words: ['OFOLEXD']]            1
      1000       [Object: 0 - words: ['OFOLEXD']]            1

[600000 rows x 2 columns]

In [80]:
single_run1_model_data

,Successful_interactions,Total_word_amount,Total_unique_word_amount
0,0,189,90
1,0,367,168
2,0,522,223
3,0,671,272
4,0,818,319
...,...,...,...
595,26329,1000,1
596,26429,1000,1
597,26529,1000,1
598,26629,1000,1


Create Graphs and Figures

In [ ]:
agent_data
model_data

#figure showing all total words over time

#figure showing all unique words over time

#figure showing all successful interactions over time

#smaller figure showing succesful interactions over time with overlaid s(t)=3t/N^2